# Community Detection

In [1]:
import networkx as nx
import numpy as np
import pandas as pd
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as tud
from tqdm import tqdm
import matplotlib.pyplot as plt
import os
import warnings
plt.rcParams['font.sans-serif'] = [u'SimHei']
plt.rcParams['axes.unicode_minus'] = False
warnings.filterwarnings("ignore")
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
np.random.seed(0)

In [2]:
import matplotlib
%matplotlib auto
matplotlib.use('Qt5Agg')

Using matplotlib backend: Qt5Agg


In [3]:
G = nx.read_edgelist('../graph/whole_undirected_graph.g')

###  Louvain algorithm

`community` is a Python library for detecting community structure in graphs. It implements Louvain algorithm to discover the best community partition. The library can also be used to visualize these algorithms.

Another famous method to detect community is the Girvan-Newman(GN) algorithm.

In the following part, We'll apply these two algorithms to our dataset.

In [4]:
import community

In [5]:
# 社区分割
part = community.best_partition(G,random_state=333)
values = [part.get(node) for node in G.nodes()]
max(values) # 划分的社区数目

30

In [6]:
# 计算模块度
mod = community.modularity(part,G)
print(mod)

0.8444919391107967


In [7]:
# 可视化(原图布局)
nx.draw_spring(G,node_color = values, node_size=20, width=0.3, alpha=0.3, with_labels=False)
plt.show() #a

In [8]:
# 可视化(社区布局) #原图布局和社区布局的区别？
V = [node for node in G.nodes()]
com_dict = {node:com for node, com in zip(V, values)}
com = [[V[i] for i in range(G.number_of_nodes()) if values[i] == j] for j in range(max(values)+1)]

G_graph = nx.Graph()
for each in com:
  G_graph.update(nx.subgraph(G, each))
color = [com_dict[node] for node in G_graph.nodes()]

pos = nx.spring_layout(G_graph, seed=4)
nx.draw(G, pos,  node_color = values, with_labels=False, node_size=20, width=0.3, alpha=0.3)
plt.show()

In [9]:
# 寻找每个子图中心度最大的节点
i = 0
for group in com:
    i += 1
    G_sub = nx.subgraph(G, group)
    degree_centrality =  nx.degree_centrality(G_sub)
    degree_centrality = sorted(degree_centrality.items(), key=lambda item: item[1], reverse=True)
    print("center for group",i,': ',degree_centrality[0])

center for group 1 :  ('赤峰南', 0.10869565217391304)
center for group 2 :  ('成都东', 0.3106796116504854)
center for group 3 :  ('广州南', 0.29464285714285715)
center for group 4 :  ('牡丹江', 0.12244897959183673)
center for group 5 :  ('南昌西', 0.30952380952380953)
center for group 6 :  ('长沙', 0.25773195876288657)
center for group 7 :  ('南宁', 0.21951219512195122)
center for group 8 :  ('太原南', 0.23333333333333334)
center for group 9 :  ('沈阳', 0.1326530612244898)
center for group 10 :  ('盐津北', 0.05172413793103448)
center for group 11 :  ('莱阳', 0.21649484536082475)
center for group 12 :  ('唐山', 0.35211267605633806)
center for group 13 :  ('阜阳', 0.2698412698412698)
center for group 14 :  ('郑州', 0.3333333333333333)
center for group 15 :  ('仲恺', 0.5882352941176471)
center for group 16 :  ('崇州', 0.9)
center for group 17 :  ('合肥南', 0.23595505617977527)
center for group 18 :  ('怀化', 0.11594202898550725)
center for group 19 :  ('月华', 0.058823529411764705)
center for group 20 :  ('西安', 0.1419753086419753)
ce

In [10]:
# 可视化(子图)
G_sub = nx.subgraph(G, com[28])
nx.draw_spring(G_sub, node_size=50, width=1, font_size=15, with_labels=True,font_color='#000000',alpha=0.3)
plt.show()

### GN算法

Girvan-Newman 算法即是一种基于介数的社区发现算法，其基本思想是根据边介数中心性(edge betweenness)从大到小的顺序不断地将边从网络中移除直到整个网络分解为各个社区。因此，Girvan-Newman 算法实际上是一种分裂方法。

Girvan-Newman 算法的基本流程如下： (1)计算网络中所有边的边介数； (2)找到边介数最高的边并将它从网络中移除； (3)重复步骤 2，直到每个节点成为一个独立的社区为止，即网络中没有边存在。

In [13]:
import copy

def all_results(G):
    results = []
    m = G.number_of_edges()
    for i in range(m):
        betweenness_dict = nx.edge_betweenness_centrality(G)
        edge_max_betweenness = max(betweenness_dict.items(), key=lambda x:x[1])[0]
        G.remove_edge(edge_max_betweenness[0], edge_max_betweenness[1])
        community = [list(subgraph) for subgraph in nx.connected_components(G)]
        community_dict = {node:0 for node in G.nodes()}
        for i in range(len(community)):
            each = community[i]
            for node in each:
                community_dict[node] = i
        results.append(community_dict)
    return results

def GN_partition(G):
    G_copy = copy.deepcopy(G)
    results = all_results(G)
    modularities = [community.modularity(results[i], G_copy) for i in range(len(results))]
    max_modularity = max(modularities)
    max_index = modularities.index(max_modularity)
    max_result = results[max_index]
    return list(max_result.values()), max_modularity

In [ ]:
# 社区分割
G_copy = copy.deepcopy(G)
gn_part = GN_partition(G_copy)
values = [gn_part.get(node) for node in G.nodes()]
max(values) # 划分的社区数目

In [13]:
gn_part

NameError: name 'gn_part' is not defined

In [ ]:
# 可视化(原图布局)
pos = nx.spring_layout(G)
nx.draw(G, pos, with_labels=False, node_size=3, width=0.5, node_color=gn_com)
plt.show()

In [ ]:
# 可视化(社区布局)
V = [node for node in G.nodes()]
com_dict = {node:com for node, com in zip(V, gn_com)}
k = max(com_dict.values()) + 1
com = [[V[i] for i in range(G.number_of_nodes()) if gn_com[i] == j] for j in range(k)]

G_graph = nx.Graph()
for each in com:
  G_graph.update(nx.subgraph(G, each))
color = [com_dict[node] for node in G_graph.nodes()]

pos = nx.spring_layout(G_graph, seed=4)
nx.draw(G, pos, with_labels=False, node_size=1, width=0.1, alpha=0.2)
nx.draw(G_graph, pos, with_labels=True, node_color=color, node_size=20, width=0.3, alpha=0.3, font_size=15, font_color='#000000')
plt.show()

In [ ]:
# 寻找每个子图中心度最大的节点
i = 0
for group in com:
    i += 1
    G_sub = nx.subgraph(G, group)
    degree_centrality =  nx.degree_centrality(G_sub)
    degree_centrality = sorted(degree_centrality.items(), key=lambda item: item[1], reverse=True)
    print("center for group",i,': ',degree_centrality[0])

In [ ]:
# 可视化(子图)
G_sub = nx.subgraph(G, com[3])
nx.draw_spring(G_sub, node_size=50, width=1, font_size=15, with_labels=True,font_color='#000000',alpha=0.3)
plt.show()

### Infomap算法（弃用）

In [ ]:
"""
def random_walk(G, pos):
    infomapWrapper = infomap.Infomap("--two-level --silent")
    for e in G.edges():
        infomapWrapper.addLink(*e)
    infomapWrapper.run()
    tree = infomapWrapper

    partition = {}
    for node in tree.iterTree():
        partition[node.physicalId] = node.moduleIndex()

    cmap = cm.get_cmap('viridis', max(partition.values()) + 1)
    nx.draw_networkx_nodes(G, pos, partition.keys(), node_size=15,
                           cmap=cmap, node_color=list(partition.values()))
    nx.draw_networkx_edges(G, pos, alpha=0.5)
    plt.show()

    modularity = community.modularity(partition, G)
    print([max(partition.values()) + 1, modularity])

    return tree.numTopModules()


infomapWrapper = infomap.Infomap("--two-level --silent")
list(G.edges())[1]
for e in G.edges():
    infomapWrapper.addLink(*e)
infomapWrapper.run()
tree = infomapWrapper

partition = {}
for node in tree.iterTree():
    partition[node.physicalId] = node.moduleIndex()

cmap = cm.get_cmap('viridis', max(partition.values()) + 1)
nx.draw_networkx_nodes(G, pos, partition.keys(), node_size=15,
                       cmap=cmap, node_color=list(partition.values()))
nx.draw_networkx_edges(G, pos, alpha=0.5)
plt.show()

modularity = community.modularity(partition, G)
print([max(partition.values()) + 1, modularity])
"""